# NER with BiLSTM + CRF Tensorflow 2 on Histo dataset

## Imports

In [ ]:
!pip install tf2crf

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/mhc/')

In [4]:
%cd "/content/drive/My Drive/Colab Notebooks/mhc/"

/content/drive/My Drive/Colab Notebooks/mhc


In [1]:
import numpy as np

from model import BilstmCrf
from text import PreProcessing

## Training

### Glove

In [ ]:
data_path = "data/histo/"

In [ ]:
embeddings = np.load("data/histo/gloveemb.npz")['embeddings']

In [ ]:
model = BilstmCrf(data_path, embeddings)

In [ ]:
model.train()

Epoch 1/25
31/31 [==============================] - 12s 377ms/step - loss: 192.7132 - accuracy: 0.8565 - val_loss: 562.9733 - val_accuracy: 0.9770
Epoch 2/25
31/31 [==============================] - 10s 324ms/step - loss: 28.8557 - accuracy: 0.9757 - val_loss: 562.2709 - val_accuracy: 0.9775
Epoch 3/25
31/31 [==============================] - 10s 325ms/step - loss: 27.1620 - accuracy: 0.9761 - val_loss: 562.0173 - val_accuracy: 0.9777
Epoch 4/25
31/31 [==============================] - 10s 317ms/step - loss: 25.5077 - accuracy: 0.9763 - val_loss: 561.9073 - val_accuracy: 0.9775
Epoch 5/25
31/31 [==============================] - 10s 320ms/step - loss: 23.6164 - accuracy: 0.9764 - val_loss: 561.6801 - val_accuracy: 0.9777
Epoch 6/25
31/31 [==============================] - 10s 318ms/step - loss: 21.8233 - accuracy: 0.9767 - val_loss: 561.4133 - val_accuracy: 0.9781
Epoch 7/25
31/31 [==============================] - 10s 318ms/step - loss: 20.2587 - accuracy: 0.9776 - val_loss: 561.1129 

In [ ]:
model.evaluate()

8/8 [==============================] - 1s 170ms/step - loss: 556.1351 - accuracy: 0.9852
Test Accuracy: 0.9852343797683716


In [ ]:
model.save("serialized/glove/")

### Histo


In [ ]:
embeddings = np.load("data/histo/glovehisto.npz")['embeddings']

In [ ]:
model = BilstmCrf(data_path, embeddings)

In [ ]:
model.train()

Epoch 1/25
31/31 [==============================] - 11s 351ms/step - loss: 217.1698 - accuracy: 0.8348 - val_loss: 620.9128 - val_accuracy: 0.9775
Epoch 2/25
31/31 [==============================] - 9s 305ms/step - loss: 28.6353 - accuracy: 0.9758 - val_loss: 620.2790 - val_accuracy: 0.9775
Epoch 3/25
31/31 [==============================] - 9s 299ms/step - loss: 27.1143 - accuracy: 0.9760 - val_loss: 620.0987 - val_accuracy: 0.9775
Epoch 4/25
31/31 [==============================] - 9s 303ms/step - loss: 25.5711 - accuracy: 0.9761 - val_loss: 619.9465 - val_accuracy: 0.9775
Epoch 5/25
31/31 [==============================] - 9s 302ms/step - loss: 24.2122 - accuracy: 0.9761 - val_loss: 619.7828 - val_accuracy: 0.9775
Epoch 6/25
31/31 [==============================] - 9s 303ms/step - loss: 22.6721 - accuracy: 0.9761 - val_loss: 619.6038 - val_accuracy: 0.9776
Epoch 7/25
31/31 [==============================] - 9s 306ms/step - loss: 21.2740 - accuracy: 0.9766 - val_loss: 619.3395 - val_

In [ ]:
model.evaluate()

8/8 [==============================] - 1s 169ms/step - loss: 614.6654 - accuracy: 0.9847
Test Accuracy: 0.9846874475479126


In [ ]:
model.save("serialized/histo/")

## Reload and predict

In [ ]:
embeddings = np.load("data/histo/gloveemb.npz")['embeddings']
model = BilstmCrf()

model.restore_model(embeddings, "serialized/glove/")

In [2]:
lines = ['The transfer began on April 9 1942 after the three-month Battle of Bataan in the Philippines during World War II',
         "Astor Pantaleón Piazzolla was an Argentine tango composer bandoneon player and arranger"]

In [ ]:
_ = model.predict(lines)

words: The transfer began  on April 9 1942 after the three-month Battle of Bataan in the Philippines during World War II
preds: O   O        B-TIME O  O     O O    O     O   O           O      O  O      O  O   O           O      O     O     


words: Astor Pantaleón Piazzolla was    an     Argentine tango  composer bandoneon player and arranger
preds: O     O         O         B-TIME I-TIME I-TIME    I-TIME                           O           




In [21]:
import spacy
from spacy import displacy

nlp = spacy.load("en_core_web_md")
doc = nlp(lines[0])

displacy.render(doc, style="ent", jupyter=True)